## Using GRIB data

In [1]:
import earthkit.data as ekd
ekd.download_example_file("test6.grib")

/Users/cgr/venv/dev/lib/python3.13/site-packages/array_api_compat/torch/_info.py:361: UserWarning: 'mkldnn' is no longer used as device type. So torch.device('mkldnn') will be deprecated and removed in the future. Please use other valid device types instead. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/c10/core/Device.cpp:42.)
  a = torch.empty((0,), device=torch.device(device_name, index=i))


In [2]:
ds = ekd.from_source("file", "test6.grib")

No GRIB data is actually loaded at this point.

### Iteration

In [3]:
for f in ds:
    print(f)

### Inspecting the contents

In [4]:
len(ds)

6

In [5]:
ds.ls()

,variable,valid_datetime,base_datetime,step,level,vertical_type,member,grid_type
0,t,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll
1,u,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll
2,v,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll
3,t,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll
4,u,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll
5,v,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll


In [6]:
ds.describe()

,,shortName,typeOfLevel,level,date,time,stepRange,number,paramId,class,stream,type,experimentVersionNumber
shortName,typeOfLevel,,,,,,,,,,,,


### Slicing

Standard Python slicing is available. It does not involve any loading/copying of GRIB data. 

In [23]:
g = ds[1]
g.ls()

,variable,valid_datetime,base_datetime,step,level,vertical_type,member,grid_type
0,u,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll


In [8]:
g = ds[1:3]
g.ls()

,variable,valid_datetime,base_datetime,step,level,vertical_type,member,grid_type
0,u,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll
1,v,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll


In [22]:
g = ds[-1]
g.ls()

,variable,valid_datetime,base_datetime,step,level,vertical_type,member,grid_type
0,v,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll


### Getting data values

#### Using values

In [10]:
v = ds[0].values
v.shape

(84,)

In [11]:
v[0:4]

array([272.56417847, 272.56417847, 272.56417847, 272.56417847])

In [12]:
v = ds.values
v.shape

(6, 84)

#### Using to_numpy()

In [13]:
v = ds[0].to_numpy()
print(v.shape)
print(ds[0].shape)

(7, 12)
(7, 12)


In [14]:
v = ds.to_numpy()
v.shape

(6, 7, 12)

### Metadata

In [15]:
ds[0].get("level")

1000

In [16]:
ds[0:2].get(["level", "variable"])

[[1000, 't'], [1000, 'u']]

In [17]:
ds.get("level")

[1000, 1000, 1000, 850, 850, 850]

The underlying ecCodes GRIB metadata keys can also be accessed but we need to use the "grib." prefix.

In [18]:
ds[0:2].get(["grib.level", "grib.shortName", "grib.centre"])

[[1000, 't', 'ecmf'], [1000, 'u', 'ecmf']]

### Selection

In [19]:
g = ds.sel(variable=["u", "v"], level=850)
g.ls()

,variable,valid_datetime,base_datetime,step,level,vertical_type,member,grid_type
0,u,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll
1,v,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll


In [20]:
g = ds.sel(variable="t")
g.ls()

,variable,valid_datetime,base_datetime,step,level,vertical_type,member,grid_type
0,t,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,1000,pressure,0,regular_ll
1,t,2018-08-01 12:00:00,2018-08-01 12:00:00,0 days,850,pressure,0,regular_ll


### Xarray

In [21]:
xds = ds.to_xarray(engine="cfgrib")
xds

KeyError: 'Key gridType not found in field'

In [ ]:
xds = ds.to_xarray(engine="cfgrib", xarray_open_dataset_kwargs={"backend_kwargs": {"ignore_keys": ["number"]}})
xds